In [1]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 4.2 MB/s 
     |████████████████████████████████| 181 kB 42.3 MB/s 
     |████████████████████████████████| 144 kB 49.2 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=488fe36e2dec8234e0e3e6f6dafce979c1a65560bd7c22302a9911df31d5627e
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [7]:
!pip install deepface

     |████████████████████████████████| 63 kB 1.1 MB/s 
     |████████████████████████████████| 2.3 MB 9.0 MB/s 
     |████████████████████████████████| 462 kB 57.3 MB/s 


In [18]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import wandb
from wandb.keras import WandbCallback
import glob
from deepface import DeepFace
import csv

In [3]:
wandb.init(project='face recognition')

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [10]:
config=wandb.config
config.learning_rate=0.001

In [9]:
embedding1=[]
for i in range(20):
   for img in glob.glob((f'./drive/MyDrive/images/behnosh_tabatabei/{i}.jpg')):
       embedding1.append(DeepFace.represent(img_path=img,model_name='ArcFace'))

embedding1=np.array(embedding1)       


arcface_weights.h5  will be downloaded to  /root/.deepface/weights/arcface_weights.h5


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/arcface_weights.h5
To: /root/.deepface/weights/arcface_weights.h5
100%|██████████| 137M/137M [00:11<00:00, 11.9MB/s]


In [11]:
embedding2=[]
for i in range(0,15):
   for img in glob.glob((f'./drive/MyDrive/images/Golshifteh_Farahani/{i}.jpg')):
       embedding2.append(DeepFace.represent(img_path=img,model_name='ArcFace'))

embedding2=np.array(embedding2)         

In [12]:
embedding4=[]
for i in range(20):
   for img in glob.glob((f'./drive/MyDrive/images/Bahram Radan/{i}.jpg')):
       embedding4.append(DeepFace.represent(img_path=img,model_name='ArcFace'))

embedding4=np.array(embedding4)         

In [14]:
embedding5=[]
for i in range(20):
   for img in glob.glob((f'./drive/MyDrive/images/shaghayegh_dehghan/{i}.jpg')):
       embedding5.append(DeepFace.represent(img_path=img,model_name='ArcFace'))

embedding5=np.array(embedding5)   
     

In [15]:
embedding=np.concatenate((embedding1,embedding2,embedding4,embedding5))

In [19]:
file=open('./drive/MyDrive/images/weights.csv','w',newline='')

with file:
    write=csv.writer(file)
    write.writerows(embedding)

In [91]:
print(embedding.shape)

(75, 512)


In [92]:
data=pd.read_csv('./drive/MyDrive/images/weights.csv')

In [93]:
X=data.iloc[:,:]
Y=data.iloc[0:,-1]
X=np.array(X)
Y=np.array(Y)


In [94]:
print(Y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]


In [95]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3)

In [96]:
Y_train=Y_train.reshape(-1,1)
Y_test=Y_test.reshape(-1,1)

In [101]:
model=tf.keras.models.Sequential([
                                  tf.keras.layers.Dense(8,activation='relu'),
                                  tf.keras.layers.Dense(16,activation='relu'),
                                  tf.keras.layers.Dense(4,activation='softmax')
                                  ])

In [102]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate),
             loss=tf.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

In [103]:
model.fit(X_train,Y_train,epochs=200,callbacks=[WandbCallback()])

Epoch 1/200
2/2 [==============================] - 0s 8ms/step - loss: 1.4591 - accuracy: 0.1176 - _timestamp: 1646473018.0000 - _runtime: 4307.0000
Epoch 2/200
2/2 [==============================] - 0s 7ms/step - loss: 1.3981 - accuracy: 0.1765 - _timestamp: 1646473018.0000 - _runtime: 4307.0000
Epoch 3/200
2/2 [==============================] - 0s 10ms/step - loss: 1.3762 - accuracy: 0.2549 - _timestamp: 1646473018.0000 - _runtime: 4307.0000
Epoch 4/200
2/2 [==============================] - 0s 11ms/step - loss: 1.3593 - accuracy: 0.2941 - _timestamp: 1646473018.0000 - _runtime: 4307.0000
Epoch 5/200
2/2 [==============================] - 0s 7ms/step - loss: 1.3477 - accuracy: 0.3922 - _timestamp: 1646473018.0000 - _runtime: 4307.0000
Epoch 6/200
2/2 [==============================] - 0s 8ms/step - loss: 1.3347 - accuracy: 0.4510 - _timestamp: 1646473018.0000 - _runtime: 4307.0000
Epoch 7/200
2/2 [==============================] - 0s 8ms/step - loss: 1.3207 - accuracy: 0.5882 - _time

In [104]:
model.evaluate(X_test,Y_test)

1/1 [==============================] - 0s 108ms/step - loss: 0.4129 - accuracy: 0.8261


[0.41293343901634216, 0.8260869383811951]